# SIFT Feature Matching

Key References
- https://docs.opencv.org/4.x/da/df5/tutorial_py_sift_intro.html
- https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html

In [ ]:
from __future__ import annotations

from typing import TYPE_CHECKING

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

import project_3d_reconstruction.mpl as proj_mpl

if TYPE_CHECKING:
    from mpl_toolkits.mplot3d.axes3d import Axes3D

# Generate simple scene

In [ ]:
x, y, z = np.indices((8, 8, 8))

cube1 = (x < 3) & (y < 3) & (z < 3)  # noqa: PLR2004
cube2 = (x >= 5) & (y >= 5) & (z >= 5)  # noqa: PLR2004
link = abs(x - y) + abs(y - z) + abs(z - x) <= 2  # noqa: PLR2004

voxelarray = cube1 | cube2 | link

colors = np.empty(voxelarray.shape, dtype=object)
colors[link] = "tab:red"
colors[cube1] = "tab:blue"
colors[cube2] = "tab:green"

ax: Axes3D
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8, 8))
ax.axis("off")
ax.voxels(voxelarray, facecolors=colors, edgecolors=colors)
fig.tight_layout()

# Save image from two perspectives.

In [ ]:
ax.view_init(elev=30, azim=-60)
image_1 = proj_mpl.fig_export_rgba(fig)

ax.view_init(elev=25, azim=-55)
image_2 = proj_mpl.fig_export_rgba(fig)

gray_1 = cv.cvtColor(image_1, cv.COLOR_BGR2GRAY)
gray_2 = cv.cvtColor(image_2, cv.COLOR_BGR2GRAY)

ifig, iaxs = plt.subplots(ncols=2, figsize=(8, 8))
iaxs[0].imshow(image_1)
iaxs[1].imshow(image_2)
ifig.tight_layout()

# Detect SIFT features

In [ ]:
sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray_1, None)
kp2, des2 = sift.detectAndCompute(gray_2, None)

image_kp_1 = cv.drawKeypoints(
    gray_1, kp1, image_1, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
image_kp_2 = cv.drawKeypoints(
    gray_2, kp2, image_2, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

ifig, iaxs = plt.subplots(ncols=2, figsize=(8, 8))
iaxs[0].imshow(image_kp_1)
iaxs[1].imshow(image_kp_2);

# Match features (FLANN)

In [ ]:
index_params = {"algorithm": 1, "trees": 5}
search_params = {"checks": 50}
flann = cv.FlannBasedMatcher(index_params, search_params)

flann_matches = flann.knnMatch(des1, des2, k=2)

match_mask = [[0, 0] for i in range(len(flann_matches))]
for i, (m, n) in enumerate(flann_matches):
    if m.distance < 0.7 * n.distance:
        match_mask[i] = [1, 0]

draw_params = {
    "matchColor": (0, 255, 0),
    "singlePointColor": (255, 0, 0),
    "matchesMask": match_mask,
    "flags": cv.DrawMatchesFlags_DEFAULT,
}
image_matches = cv.drawMatchesKnn(
    image_1, kp1, image_2, kp2, flann_matches, None, **draw_params
)

ifig, iax = plt.subplots()
iax.imshow(image_matches)
ifig.tight_layout()

# Match features (brute-force)

In [ ]:
bf = cv.BFMatcher()
bf_matches = bf.knnMatch(des1, des2, k=2)

good = []
for m, n in bf_matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])

image_matches = cv.drawMatchesKnn(
    image_1,
    kp1,
    image_2,
    kp2,
    good,
    None,
    flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)


ifig, iax = plt.subplots()
iax.imshow(image_matches)
ifig.tight_layout()